<a href="https://colab.research.google.com/github/Rocking-Priya/704-fall-projects-2025/blob/main/project_(10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DX 704 Week 10 Project
In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [1]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

Cloning into 'SQuAD-explorer'...
remote: Enumerating objects: 5563, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 5563 (delta 11), reused 17 (delta 6), pack-reused 5539 (from 1)
Receiving objects: 100% (5563/5563), 52.26 MiB | 30.72 MiB/s, done.
Resolving deltas: 100% (3563/3563), done.


In [2]:
import json

In [3]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [4]:
type(train_data)

dict

In [5]:
list(train_data.keys())

['data', 'version']

In [6]:
type(train_data["data"])

list

In [7]:
len(train_data["data"])

442

In [8]:
type(train_data["data"][0])

dict

In [9]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [10]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [11]:
len(train_data["data"][0]["paragraphs"])

55

In [12]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [13]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [14]:
# YOUR CHANGES HERE


import pandas as pd
from pathlib import Path

# paths (adjust if your repo is located elsewhere)
input_path = Path("SQuAD-explorer/dataset/train-v1.1.json")
output_path = Path("parsed.tsv")

# 1) Load JSON
with open(input_path, "r", encoding="utf-8") as f:
    squad = json.load(f)

# 2) Build list of rows: each row = (document_title, paragraph_index, paragraph_context)
rows = []
for doc in squad.get("data", []):
    title = doc.get("title", "")
    paragraphs = doc.get("paragraphs", [])
    for i, para in enumerate(paragraphs):            # i is zero-indexed as required
        context = para.get("context", "")
        rows.append({
            "document_title": title,
            "paragraph_index": i,                     # zero-indexed
            "paragraph_context": context
        })

# 3) Create DataFrame
df = pd.DataFrame(rows, columns=["document_title", "paragraph_index", "paragraph_context"])

# Quick sanity checks (optional but recommended)
print("Number of documents in JSON:", len(squad.get("data", [])))
print("Total paragraphs (rows):", len(df))
# Uncomment next line if you want to verify it matches the sum shown in the notebook (18896)
# assert len(df) == 18896, f"Expected 18896 paragraphs but found {len(df)}"

# 4) Write TSV safely (pandas handles quotes/newlines)
df.to_csv(output_path, sep="\t", index=False, encoding="utf-8")

print(f"Wrote TSV to: {output_path.resolve()}")
# Show the first 3 rows to visually confirm
print(df.head(3).to_string(index=False))


Number of documents in JSON: 442
Total paragraphs (rows): 18896
Wrote TSV to: /content/parsed.tsv
          document_title  paragraph_index                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

Submit "parsed.tsv" in Gradescope.

In [20]:
from google.colab import files
#files.download("parsed.tsv")

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [18]:
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import numpy as np # Added numpy import

# Paths
parsed_path = Path("/content/parsed.tsv")   # adjust if needed
out_vectors = Path("/content/paragraph_vectors.npy")
out_meta = Path("/content/paragraph_meta.csv")

# 1) load paragraphs
df = pd.read_csv(parsed_path, sep="\t", encoding="utf-8")
texts = df["paragraph_context"].fillna("").astype(str).tolist()  # ensure strings

# 2) load model
# Choose a model that produces 1024-dimensional embeddings
model_name = "all-roberta-large-v1"  # Outputs 1024-d embeddings
model = SentenceTransformer(model_name)

# 3) compute embeddings (batching is handled by the model)
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True)  # shape (n_paragraphs, D)

print("embeddings shape:", embeddings.shape)

# 4) if embeddings are not 1024-d, project to 1024 using PCA (deterministic)
target_dim = 1024
if embeddings.shape[1] != target_dim:
    print(f"Projecting from {embeddings.shape[1]} -> {target_dim} using PCA (fit on all paragraphs).")
    pca = PCA(n_components=target_dim, random_state=0)
    embeddings_1024 = pca.fit_transform(embeddings)
else:
    embeddings_1024 = embeddings

print("final shape:", embeddings_1024.shape)

# 5) optional: L2-normalize for cosine-similarity retrieval
from sklearn.preprocessing import normalize
embeddings_1024 = normalize(embeddings_1024, norm='l2', axis=1)

# 6) save results
np.save(out_vectors, embeddings_1024)            # saved as float32/pickle-free .npy
df.to_csv(out_meta, index=False)                 # metadata: title, index, context (if you want)

print("Saved:", out_vectors, out_meta)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/296 [00:00<?, ?it/s]

embeddings shape: (18896, 1024)
final shape: (18896, 1024)
Saved: /content/paragraph_vectors.npy /content/paragraph_meta.csv


In [ ]:
!pip uninstall -y sentence-transformers transformers
!pip install -q --upgrade sentence-transformers transformers scikit-learn

Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [19]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
import json
from pathlib import Path

# Paths (adjust if your files are located elsewhere)
out_vectors = Path("/content/paragraph_vectors.npy")
out_meta = Path("/content/paragraph_meta.csv")
output_tsv_gz = Path("paragraph-vectors.tsv.gz")

# Load the metadata dataframe
df_meta = pd.read_csv(out_meta, encoding="utf-8")

# Load the embeddings
embeddings_1024 = np.load(out_vectors)

# Convert embeddings to a list of JSON strings
# Each row of embeddings_1024 is a vector, convert it to a Python list and then to a JSON string
df_meta["paragraph_vector_json"] = [json.dumps(vec.tolist()) for vec in embeddings_1024]

# Select and reorder columns as required
final_df = df_meta[["document_title", "paragraph_index", "paragraph_vector_json"]]

# Save to gzipped TSV file
# pandas to_csv with .gz extension automatically handles gzip compression
final_df.to_csv(output_tsv_gz, sep="\t", index=False, encoding="utf-8", compression='gzip')

print(f"Saved paragraph vectors to: {output_tsv_gz.resolve()}")
print(f"First 3 rows of {output_tsv_gz.name}:")
# Display the head of the DataFrame to confirm
print(final_df.head(3).to_string(index=False))

Saved paragraph vectors to: /content/paragraph-vectors.tsv.gz
First 3 rows of paragraph-vectors.tsv.gz:
          document_title  paragraph_index                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

Submit "paragraph-vectors.tsv.gz" in Gradescope.

In [21]:
files.download("paragraph-vectors.tsv.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [22]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np
import json
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize



In [23]:
# Path to the questions file
questions_path = Path("/content/questions.tsv")  # adjust if different
df_q = pd.read_csv(questions_path, sep="\t", encoding="utf-8")

# See what columns exist
print(df_q.head())


   question_id                                           question
0            1  What was the goal of the abuse of region project?
1            4  How many satellites in the Beidou-1 constellat...
2            7  When did Beyoncé  receive ten nominations for ...
3           10  With which goddess did Sulla, Pompey, and Juli...
4           13  What area is considered to have a desert clima...


In [24]:
model_name = "all-roberta-large-v1"
model = SentenceTransformer(model_name)


In [26]:
texts = df_q["question"].fillna("").astype(str).tolist()

embeddings = model.encode(texts, batch_size=64, show_progress_bar=True)
print("Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Embeddings shape: (100, 1024)


In [27]:
target_dim = 1024
if embeddings.shape[1] != target_dim:
    print(f"Projecting from {embeddings.shape[1]} -> {target_dim} using PCA.")
    pca = PCA(n_components=target_dim, random_state=0)
    embeddings_1024 = pca.fit_transform(embeddings)
else:
    embeddings_1024 = embeddings


In [28]:
embeddings_1024 = normalize(embeddings_1024, norm='l2', axis=1)


In [29]:
df_q["question_vector_json"] = [json.dumps(vec.tolist()) for vec in embeddings_1024]


In [30]:
final_q = df_q[["question_id", "question_vector_json"]]


Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [32]:
output_file = Path("/content/question-vectors.tsv")
final_q.to_csv(output_file, sep="\t", index=False, encoding="utf-8")

print(f"Saved question vectors to: {output_file.resolve()}")
print(final_q.head(3).to_string(index=False))

Saved question vectors to: /content/question-vectors.tsv
 question_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

Submit "question-vectors.tsv" in Gradescope.

In [33]:
files.download("question-vectors.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [34]:
# YOUR CHANGES HERE
from sklearn.neighbors import NearestNeighbors
para_path = Path("/content/paragraph-vectors.tsv.gz")
df_para = pd.read_csv(para_path, sep="\t", compression="gzip", encoding="utf-8")

# Convert JSON string back to numeric arrays
para_vectors = np.vstack(df_para["paragraph_vector_json"].apply(json.loads).to_numpy())

print("Paragraph vectors shape:", para_vectors.shape)


Paragraph vectors shape: (18896, 1024)


In [35]:
ques_path = Path("/content/question-vectors.tsv")
df_ques = pd.read_csv(ques_path, sep="\t", encoding="utf-8")

# Convert JSON strings back to numeric arrays
ques_vectors = np.vstack(df_ques["question_vector_json"].apply(json.loads).to_numpy())

print("Question vectors shape:", ques_vectors.shape)


Question vectors shape: (100, 1024)


In [36]:
nn = NearestNeighbors(n_neighbors=5, metric="euclidean")
nn.fit(para_vectors)


,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'euclidean'
,p,2
,metric_params,None
,n_jobs,None


In [37]:
distances, indices = nn.kneighbors(ques_vectors)


In [38]:
rows = []
for q_idx, q_id in enumerate(df_ques["question_id"]):
    for rank, para_idx in enumerate(indices[q_idx], start=1):  # ranks 1..5
        para_row = df_para.iloc[para_idx]
        rows.append({
            "question_id": q_id,
            "question_rank": rank,
            "document_title": para_row["document_title"],
            "paragraph_index": para_row["paragraph_index"]
        })

df_matches = pd.DataFrame(rows, columns=["question_id", "question_rank", "document_title", "paragraph_index"])


Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [39]:
# YOUR CHANGES HERE

output_path = Path("/content/question-matches.tsv")
df_matches.to_csv(output_path, sep="\t", index=False, encoding="utf-8")

print(f"Saved matches to: {output_path.resolve()}")
print(df_matches.head(10).to_string(index=False))


Saved matches to: /content/question-matches.tsv
 question_id  question_rank                          document_title  paragraph_index
           1              1                                Genocide                9
           1              2 Affirmative_action_in_the_United_States                2
           1              3                        Marshall_Islands               24
           1              4                             Great_power                3
           1              5                    Political_corruption               16
           4              1      BeiDou_Navigation_Satellite_System               12
           4              2      BeiDou_Navigation_Satellite_System               18
           4              3      BeiDou_Navigation_Satellite_System                1
           4              4      BeiDou_Navigation_Satellite_System               14
           4              5      BeiDou_Navigation_Satellite_System               10


Submit "question-matches.tsv" in Gradescope.

In [40]:
files.download("question-matches.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


In [41]:
df_matches = pd.read_csv("/content/question-matches.tsv", sep="\t")

In [42]:
first_five_questions = sorted(df_matches["question_id"].unique())[:5]


In [44]:
df_worst = df_matches[
    (df_matches["question_id"].isin(first_five_questions)) &
    (df_matches["question_rank"] == 5)
]



In [45]:
df_worst = df_worst[["question_id", "document_title", "paragraph_index"]]


In [46]:
output_path = Path("/content/worst-paragraphs.tsv")
df_worst.to_csv(output_path, sep="\t", index=False, encoding="utf-8")

print(f"Saved to: {output_path.resolve()}")
print(df_worst)


Saved to: /content/worst-paragraphs.tsv
    question_id                      document_title  paragraph_index
4             1                Political_corruption               16
9             4  BeiDou_Navigation_Satellite_System               10
14            7                             Beyoncé               24
19           10            Religion_in_ancient_Rome                8
24           13                               Tibet               26


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.